## Imports

In [83]:
import os
import asyncio
from pydantic import BaseModel, Field
from typing import List
from langgraph_sdk import get_client
from dotenv import load_dotenv
from supabase import create_client, Client

# Load environment variables
load_dotenv()


True

## Authentication

In [84]:
# Supabase configuration
SUPABASE_URL = os.environ.get("SUPABASE_URL")
SUPABASE_KEY = os.environ.get("SUPABASE_KEY")
USER_EMAIL = os.environ.get("USER_EMAIL")
USER_PASSWORD = os.environ.get("USER_PASSWORD")

# LangGraph configuration
SUPERVISOR_URL = "http://localhost:2025"  # Supervisor agent port                                                                                                                      │ │
TOOLS_AGENT_URL = "http://localhost:2024"  # Sub-agents port 

In [85]:
def authenticate_supabase() -> str:
    """Authenticate with Supabase and return access token"""
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    response = supabase.auth.sign_in_with_password({
        "email": USER_EMAIL,
        "password": USER_PASSWORD,
    })
    if response.session is None:
        raise RuntimeError(f"Supabase authentication failed: {response}")
    return response.session.access_token

def get_authenticated_client():
    """Get authenticated LangGraph client"""
    access_token = authenticate_supabase()
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}",
    }
    return get_client(url=SUPERVISOR_URL, headers=headers)

In [86]:
client = get_authenticated_client()
print(client)

## Create/Update Schema

In [87]:
# --- Sub-Component for Research Findings ---
class FactEntry(BaseModel):
    """A single piece of factual or researched information."""
    category: str = Field(..., description="The broad topic or category this fact belongs to (e.g., 'Historical Context', 'Current Statistics', 'Key Definition').")
    detail: str = Field(..., description="The specific, verified fact or piece of data provided by the Research Assistant.")

# --- Sub-Component for Analytical Conclusions ---
class InsightEntry(BaseModel):
    """A key conclusion or interpretation derived from the facts."""
    focus: str = Field(..., description="The specific area of the original request this insight addresses (e.g., 'Causality', 'Implications', 'Future Trends').")
    conclusion: str = Field(..., description="The derived meaning, relationship, or conclusion based on the research data.")


# --- Main Supervisor Response Schema ---
class IntegratedProjectResponse(BaseModel):
    """
    The final, unified response generated by the Supervisor after integrating research and analysis.
    """
    original_query_summary: str = Field(..., description="A brief restatement of the user's initial request.")
    research_summary: List[FactEntry] = Field(description="The core factual data points that were collected by the Research Assistant.")
    analysis_report: List[InsightEntry] = Field(description="The key interpretations and conclusions drawn by the Analysis Assistant.")
    final_answer: str = Field(..., description="The comprehensive, synthesized final answer that directly addresses all parts of the original query.")

schema_name = "IntegratedProjectResponse"

In [88]:
# Get user info to build namespace
access_token = authenticate_supabase()
supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
user_response = supabase.auth.get_user(access_token)
user_id = user_response.user.id


In [89]:
await client.store.put_item([user_id, "schemas"], schema_name, IntegratedProjectResponse.model_json_schema())
print(f"Stored schema: {schema_name}")

Stored schema: IntegratedProjectResponse


In [90]:
IntegratedProjectResponse.model_json_schema()

{'$defs': {'FactEntry': {'description': 'A single piece of factual or researched information.',
   'properties': {'category': {'description': "The broad topic or category this fact belongs to (e.g., 'Historical Context', 'Current Statistics', 'Key Definition').",
     'title': 'Category',
     'type': 'string'},
    'detail': {'description': 'The specific, verified fact or piece of data provided by the Research Assistant.',
     'title': 'Detail',
     'type': 'string'}},
   'required': ['category', 'detail'],
   'title': 'FactEntry',
   'type': 'object'},
  'InsightEntry': {'description': 'A key conclusion or interpretation derived from the facts.',
   'properties': {'focus': {'description': "The specific area of the original request this insight addresses (e.g., 'Causality', 'Implications', 'Future Trends').",
     'title': 'Focus',
     'type': 'string'},
    'conclusion': {'description': 'The derived meaning, relationship, or conclusion based on the research data.',
     'title': '

## Read schemas

In [91]:
results = await client.store.search_items([user_id, "schemas"])
schemas = [item["key"] for item in results["items"]]
print("Available schemas:")
for schema in schemas:
    print(f"  - {schema}")


Available schemas:
  - RecipeNutritionAnalysis
  - IngredientClassification
  - FoodSafetyReport
  - MenuPlanning
  - RecipeInstructions
  - SupplierQuote
  - QualityInspection
  - AllergyAnalysisResponse
  - IntegratedProjectResponse


## Invoking agent with schema

In [92]:
# Sub-agent IDs (created on tools agent port 2024)                                                                                                                                     │ │
SUB_AGENT_1_ID = "d45f6b26-17a5-4c73-95a0-43d41f4d97a7"  # Research assistant                                                                                                          │ │
SUB_AGENT_2_ID = "92deb9bc-7f8c-4dc2-b07e-6d1a92e3b3bb"  # Analysis assistant 
SUPERVISOR_GRAPH_ID = "multi_agent_supervisor"

In [93]:
# Test recipe for allergy analysis
TEST_PROMPT = """
Please prepare a report detailing the three most significant differences between the historical economic models of the 1950s (post-WWII boom) and the current leading economic models of 2020s. Based on this comparison, analyze and provide a conclusion on which era's models were more effective at long-term wealth distribution across social classes, and why.
"""

In [94]:
assistant=await client.assistants.create(
    graph_id = SUPERVISOR_GRAPH_ID,
    config={
        "configurable": {
            "x-supabase-access-token": access_token,
            # Supervisor-specific required fields
            "agents": [
                {
                    "deployment_url": TOOLS_AGENT_URL,
                    "agent_id": SUB_AGENT_1_ID,
                    "name": "research_assistant"  # Used in delegate_to_research_assistant
                },
                {
                    "deployment_url": TOOLS_AGENT_URL,
                    "agent_id": SUB_AGENT_2_ID,
                    "name": "analysis_assistant"  # Used in delegate_to_analysis_assistant
                }
            ],
            "system_prompt": "You are a supervisor AI coordinating specialist agents. Delegate tasks to research_assistant for information gathering and analysis_assistant for breaking down complex topics.",
            "supervisor_model": "openai:gpt-4o-mini",
            "OutputSchemaName": "IntegratedProjectResponse",  # Your schema name
            
        },
        
    },
    name="Structured Output Supervisor"
)

print("Assistant created successfully")
print(f"Assistant ID: {assistant['assistant_id']}")
print(f"Assistant Name: {assistant['name']}")
print(f"Schema: {assistant['config']['configurable']['OutputSchemaName']}")
print(f"Version: {assistant['version']}")



Assistant created successfully
Assistant ID: 28fd5714-5a87-4452-9a30-8c6fc5b53ca5
Assistant Name: Structured Output Supervisor
Schema: IntegratedProjectResponse
Version: 1


In [95]:
# Create a new thread
thread = await client.threads.create()

print(f"Thread created:{thread['thread_id']}")

Thread created:e09ab5ee-82f9-4fdc-bb41-83296a9abbe2


In [96]:
input_data = {"messages": [{"role": "user", "content": TEST_PROMPT}]}

In [97]:
import json

# Store the final structured response
final_response = None

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant["assistant_id"],
    input=input_data,
    stream_mode="values",
):
    if chunk.event == "metadata":
        print(f"📋 Run started (ID: {chunk.data.get('run_id', 'Unknown')[:8]}...)\n")

    elif chunk.event == "values":
        # Keep updating with latest structured response
        if "structured_response" in chunk.data:
            final_response = chunk.data["structured_response"]

# Print only the final result
if final_response:
    print("=" * 80)
    print("🎯 Structured Output (JSON):")
    print("=" * 80)
    print(json.dumps(final_response, indent=2))
    print()

📋 Run started (ID: 1f09f169...)

🎯 Structured Output (JSON):
{
  "original_query_summary": "Comparison of economic models from the 1950s and the 2020s, focusing on wealth distribution across social classes.",
  "research_summary": [
    {
      "category": "Economic Growth Drivers",
      "detail": "The 1950s were characterized by industrial expansion driven by manufacturing and consumer demand, while the 2020s are driven by technology, globalization, and a service-oriented economy."
    },
    {
      "category": "Labor Market Dynamics",
      "detail": "The 1950s featured a stable labor market dominated by manufacturing jobs and strong unions, whereas the 2020s see a rise in gig economy jobs and less job stability due to automation."
    },
    {
      "category": "Wealth Distribution and Social Equity",
      "detail": "The 1950s had relatively high income equality with strong middle class growth, contrasted with significant income inequality and wealth concentration in the 2020s."


## Delete Schema

In [98]:
schema_name_to_delete = "IntegratedProjectResponse"

In [99]:
# await client.store.delete_item([user_id, "schemas"], schema_name_to_delete)
print(f"Deleted schema: {schema_name}")

Deleted schema: IntegratedProjectResponse
